In [ ]:
# here only executting stuff. no functions or classes

In [ ]:
## um die models und functions hier nutzen zu können, muss man die datein als module importieren. 
## das geht auch wenn das notebeooks sind, ist aber deutlich einfacher, wenn es einfach .py dateien sind.

In [ ]:
!pip install -r requirements.txt

## CycleGAN Maps 


In [55]:
#!/usr/bin/env python3
#-*- coding:utf-8 -*-
#$ -l cuda=1 # remove this line when no GPU is needed!
#$ -q all.q # do not fill the qlogin queue
#$ -cwd # start processes in current working directory
#$ -V # provide environment variables to processes
#$ -t 1-8 # start 8 instances: to train different models in parallel
#Cluster settings, 
'''
try:
    model_param_id = int(os.environ['SGE_TASK_ID'])
except:
    print("no SGE_TASK_ID set, choosing default model parameters ")
    model_param_id = 0 #param_train_cycle_list[0] should be default model params
'''
import importlib
import models
import functions 
import torch
import torch.nn as nn
importlib.reload(models)
importlib.reload(functions)
import numpy as np
#different import way for cluster
'''
import imp
models = imp.load_source('models', './models.py')
functions = imp.load_source('functions', './functions.py')
'''

# init CycleGAN
genA2B = models.Generator(input_nc=3, output_nc=3, n_residual_blocks=9)
genB2A = models.Generator(input_nc=3, output_nc=3, n_residual_blocks=9)
discA  = models.Discriminator(input_nc=3)
discB  = models.Discriminator(input_nc=3)
classifier = models.Classifier().net
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
databaseName = "maps"
root_path_data = "./data/"+databaseName
root_path_checkpoints = "./checkpoints/"+databaseName

targetEpoch = 50
saveAt = 1 #how often to store the model at training
print("the device ist at")
print(device)

if device == torch.device("cuda"):
    #debug
    print("should models to cuda")
    print("device == cuda -> "+ str(device == "cuda:0"))
    #model.load_state_dict(torch.load("{}/{} epoch{}".format(path, name, epoch)))
else:
    #debug
    print("should models to cpu")
    print("device == cuda -> "+ str(device == "cuda:0"))
    #model.load_state_dict(torch.load("{}/{} epoch{}".format(path, name, epoch),map_location=torch.device('cpu')))

cycle  = models.CycleGAN(genA2B, genB2A, discA, discB, classifier, device, root_path_data, root_path_checkpoints)


param_train1  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r9_advMSE_l10", resnet_blocks=9, loss_adv=torch.nn.MSELoss(), lambdas=(10,0.5)) # default
param_train2  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r9_advMSE_l5" , resnet_blocks=9, loss_adv=torch.nn.MSELoss(), lambdas=(5,0.5))
param_train3  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r9_advL1_l10", resnet_blocks=9, loss_adv=torch.nn.L1Loss(), lambdas=(10,0.5))
param_train4  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r9_advL1_l5" , resnet_blocks=9, loss_adv=torch.nn.L1Loss(), lambdas=(5,0.5))
param_train5  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r5_advMSE_l10", resnet_blocks=5, loss_adv=torch.nn.MSELoss(), lambdas=(10,0.5))
param_train6  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r5_advMSE_l5" , resnet_blocks=5, loss_adv=torch.nn.MSELoss(), lambdas=(5,0.5))
param_train7  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r5_advL1_l10", resnet_blocks=5, loss_adv=torch.nn.L1Loss(), lambdas=(10,0.5))
param_train8  = models.Param(channels = 3, epochs = targetEpoch, saveEpoch = saveAt, size= 256,  name ="cycle_r5_advL1_l5" , resnet_blocks=5, loss_adv=torch.nn.L1Loss(), lambdas=(5,0.5))
param_train_cycle_list = [param_train1, param_train2, param_train3, param_train4, param_train5, param_train6, param_train7, param_train8]

# paramter to load the background error verison. since this extension needed major changes in training/loaded training was done locally.
param_back_err = models.Param(name ="background_default", epochs = 80)

# paramters needed to call eval_testset() function
param_eval_testset = models.Param(channels = 3, size= 256) 

# parameters used for training the classifier
param_train_classifier = models.Param(channels = 3, epochs = targetEpoch, size = 256, name = "classifier_test")

the device ist at
cuda
should models to cuda
device == cuda -> False


## Generate Results

In [56]:
#for every model --TODO
#for paramItem in param_train_cycle_list:
    #load model
    #cycle.load_cycle_nets(epoch = 50, model_name = paramItem.name)
    #print(device)
    #generate for every test image some output image
    #cycle._create_evalset_paired(paramItem,subfolder=paramItem.name)
    #calculate error with expected image

#compare errors between models

#run pix2pix ? and compare it with that?


## Calculate Error for each model

In [57]:
#MSE? SSIM? FCN? 
import os
from PIL import Image
import torchvision.transforms as transforms


losses= {}
for paramItem in param_train_cycle_list:
    source_domains = ["A", "B"]
    subfolder = paramItem.name
    losses[paramItem.name]={}
    for source in source_domains:  
        losses[paramItem.name][source]=[]
        folder=root_path_data + "/eval{}/{}/".format(source,subfolder)
        folderSize = int(len(os.listdir(folder)) / 3)
        for i in range(0,folderSize):
            gen="{}{}_generated.jpg".format(folder, i)
            exp="{}{}_expected.jpg".format(folder, i)
            gen=Image.open(gen)
            exp=Image.open(exp)
            loss=functions.calcL1(gen,exp)
            losses[paramItem.name][source].append(loss)
    

## display losses

In [73]:
for paramItem in param_train_cycle_list:
    source_domains = ["A", "B"]
    for source in source_domains:
        summ = np.array(losses[paramItem.name][source]).sum()
        std = np.std(losses[paramItem.name][source])
        overall = np.array(losses[paramItem.name]['A']).sum() + np.array(losses[paramItem.name]['B']).sum()
        print("param:{} source:{} std:{:2f} sum:{:0f} overall:{}".format(paramItem.name,source,std,summ,overall))

param:cycle_r9_advMSE_l10 source:A std:0.021128 sum:44.468000 overall:207.642
param:cycle_r9_advMSE_l10 source:B std:0.027676 sum:163.174000 overall:207.642
param:cycle_r9_advMSE_l5 source:A std:0.018349 sum:41.879000 overall:212.426
param:cycle_r9_advMSE_l5 source:B std:0.027998 sum:170.547000 overall:212.426
param:cycle_r9_advL1_l10 source:A std:0.020923 sum:43.669000 overall:201.413
param:cycle_r9_advL1_l10 source:B std:0.026593 sum:157.744000 overall:201.413
param:cycle_r9_advL1_l5 source:A std:0.017752 sum:45.098000 overall:202.21200000000002
param:cycle_r9_advL1_l5 source:B std:0.026540 sum:157.114000 overall:202.21200000000002
param:cycle_r5_advMSE_l10 source:A std:0.023278 sum:45.731000 overall:211.471
param:cycle_r5_advMSE_l10 source:B std:0.028075 sum:165.740000 overall:211.471
param:cycle_r5_advMSE_l5 source:A std:0.021359 sum:44.727000 overall:205.86300000000003
param:cycle_r5_advMSE_l5 source:B std:0.027667 sum:161.136000 overall:205.86300000000003
param:cycle_r5_advL1_l10

In [ ]:
bla
for paramItem in param_train_cycle_list:
    print(type(paramItem.name))
    print(type("hallo"))

8

In [ ]:
hallo["asd"]=123